<font size=5>Prepare the data</font>

In [3]:
# import packages
import pandas as pd
import numpy as np
import datetime
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings("ignore")

read table customer_demographic

In [4]:
# read table
xls = pd.ExcelFile(r"D:\XIAZAI\KPMG_Module1\KPMG_Module1\KPMG_VI_New_raw_data_update_final.xlsx")
df = pd.read_excel(xls, 'CustomerDemographic')

Next, we clean the data based on Task1

In [8]:
# drop 'default' column
df = df.drop('default', axis = 1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
customer_id                            4000 non-null int64
first_name                             4000 non-null object
last_name                              3875 non-null object
gender                                 4000 non-null object
past_3_years_bike_related_purchases    4000 non-null int64
DOB                                    3913 non-null datetime64[ns]
job_title                              3494 non-null object
job_industry_category                  3344 non-null object
wealth_segment                         4000 non-null object
deceased_indicator                     4000 non-null object
owns_car                               4000 non-null object
tenure                                 3913 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 375.1+ KB


In [11]:
# replace inconsistent values in gender column
df['gender'] = df['gender'].replace('F', 'Female').replace('Femal', 'Female').replace('M', 'Male').replace('U', 'Unspecified')

In [38]:
df['gender'].value_counts

<bound method IndexOpsMixin.value_counts of 0       0
1       1
2       1
3       1
4       0
       ..
3995    0
3996    0
3997    2
3998    1
3999    1
Name: gender, Length: 4000, dtype: int64>

<font size=5>Exploring the data</font>

before we train our data, we need to explore the distribution of data to ensure our model be more confident.

In [40]:
df['gender'].describe()

count    4000.000000
mean        0.512250
std         0.542147
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         2.000000
Name: gender, dtype: float64

In [43]:
df['tenure'].describe()

count    4000.000000
mean        9.871500
std         6.693817
min        -1.000000
25%         4.000000
50%        10.000000
75%        15.000000
max        21.000000
Name: tenure, dtype: float64

In [44]:
df['job_industry_category'].describe()

count    4000.000000
mean        2.426000
std         2.762233
min        -1.000000
25%         0.000000
50%         1.000000
75%         6.000000
max         8.000000
Name: job_industry_category, dtype: float64

it looks the data distributes ok.

we want to know which feature is imortant to predict bike-related purchase to find our target customers, so we would use RandomForest Model to train the dataset.

<font size=5>Feature Engnieering</font>

first, we choose a list of features and factorize the non-numeric feature to better use RandomForest model.

In [14]:
# factorize features
factorizeFeatures = ['gender', 'job_industry_category', 'owns_car', 'tenure', 'wealth_segment']
for feature in factorizeFeatures:
    labels, uniques = pd.factorize(df[feature].values)
    print(labels, uniques)
    df[feature] = labels

[0 1 1 ... 2 1 1] ['Female' 'Male' 'Unspecified']
[ 0  1  2 ...  3  6 -1] ['Health' 'Financial Services' 'Property' 'IT' 'Retail' 'Argiculture'
 'Manufacturing' 'Telecommunications' 'Entertainment']
[0 0 0 ... 1 0 1] ['Yes' 'No']
[ 0  1  2 ... -1 21  0] [11. 16. 15.  7.  8. 13. 20.  9.  6.  1. 18. 21. 12. 19. 14.  4. 22.  5.
 17.  2.  3. 10.]
[0 0 0 ... 2 1 1] ['Mass Customer' 'Affluent Customer' 'High Net Worth']


Next, We split the dataset into training set and testing set and trained with RandomForest Classifier. RandomForest will randomly selected these features to build a decision tree.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
x = df.loc[:, factorizeFeatures].values
y = df.loc[:, 'past_3_years_bike_related_purchases'].values
print(x)

[[ 0  0  0  0  0]
 [ 1  1  0  1  0]
 [ 1  2  0  2  0]
 ...
 [ 2  3  1 -1  2]
 [ 1  6  0 21  1]
 [ 1 -1  1  0  1]]


In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 0)

In [19]:
# set parameters
forest = RandomForestClassifier(n_estimators = 300, max_depth = 2, random_state = 0, n_jobs = -1)
forest.fit(x_train, y_train.astype('int'))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [20]:
# calculate feature importance
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) % - *s %f" % (f + 1, 30, factorizeFeatures[indices[f]], importances[indices[f]]))

 1) tenure                         0.336880
 2) job_industry_category          0.243554
 3) wealth_segment                 0.209346
 4) gender                         0.153458
 5) owns_car                       0.056761


the output indicates the importance of features so we can choose good features to analyze.And tenure, job_industry_category, wealth_segment are good features to presict bike purchases thus find target customers to drive the most value.

Next, we would use Excel or Tableau to visualize and analyze the good features and bike purchases, or we could adapt models like RFM to verify our hypothesis.